In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import findspark
findspark.init("/home/rajdeep/spark-3.5.0-bin-hadoop3/")

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [16]:
#initalizing the spark object
spark = SparkSession.builder.appName("movie_rec").getOrCreate()

In [5]:
#reading the data into dataframe
df = spark.read.csv("data/movielens_ratings.csv",header=True,inferSchema=True)

In [6]:
df.columns

['movieId', 'rating', 'userId']

In [7]:
# splitting the available data into train - test df
train_df, test_df = df.randomSplit([0.7,0.3])

In [8]:
#initializing the ALS for recommender model
als = ALS(maxIter=10, regParam=0.1, userCol='userId',itemCol='movieId',ratingCol='rating')

In [9]:
#fitting the model's on train data
model = als.fit(train_df)

23/12/20 18:28:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/20 18:28:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
# transforming the test data
prediction = model.transform(test_df)

In [11]:
#initializing the evaluator to understand the acc of the model
evaluator = RegressionEvaluator(labelCol='rating',predictionCol='prediction', metricName='rmse')

In [12]:
#getting rmse score
evaluator.evaluate(prediction)

1.1102764045435518

In [13]:
#creating df with single user data to predict it recommended movies
single_user_pred = df.filter(df.userId == 11)

In [14]:
#predicting themovie rating to decide which movie to recommend
single_user_pred = model.transform(single_user_pred)

In [15]:
#printing  the movie from most to least likely to be like by user
single_user_pred.orderBy('prediction', ascending=False).show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     30|   5.0|    11| 4.8654146|
|     32|   5.0|    11|   4.82667|
|     18|   5.0|    11|  4.514018|
|     48|   5.0|    11| 4.4334364|
|     27|   5.0|    11| 4.3876143|
|     79|   5.0|    11|  4.307012|
|     69|   5.0|    11| 4.2936707|
|     13|   4.0|    11| 3.7172916|
|     38|   4.0|    11| 3.5677319|
|     23|   5.0|    11| 3.5188491|
|     19|   4.0|    11| 3.4875712|
|     90|   4.0|    11|  3.459093|
|     51|   3.0|    11| 3.4172406|
|     66|   4.0|    11| 3.4066904|
|     35|   3.0|    11|   3.00107|
|     50|   4.0|    11| 2.9465716|
|     80|   3.0|    11|  2.860137|
|     72|   3.0|    11| 2.5653956|
|     71|   3.0|    11| 2.5652914|
|     36|   2.0|    11| 2.4310327|
+-------+------+------+----------+
only showing top 20 rows

